In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import numpy as np

import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import f1
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

## Data Loading

In [2]:
import helper
data = helper.load_PAMAP2()

from dataset import TimeSeriesDataset
dtset = TimeSeriesDataset(data)

weight = helper.dist(dtset, 7)

train, val = helper.split(dtset, val_pc = 0.25)
train_weight = helper.dist(train, 7)
val_weight = helper.dist(val, 7)
print(train_weight, val_weight)
train_iter = torch.utils.data.DataLoader(train, batch_size = 32, shuffle = True, num_workers = 8, pin_memory = True)
val_iter = torch.utils.data.DataLoader(val, batch_size = 32, num_workers = 8, pin_memory = True)

File exists. Loading
Windowing
Done!
[[0.15550523]
 [0.14827995]
 [0.15539739]
 [0.18516122]
 [0.07753694]
 [0.12757468]
 [0.15054459]] [[0.15436893]
 [0.15145631]
 [0.14401294]
 [0.18640777]
 [0.07540453]
 [0.14433657]
 [0.14401294]]


## Training DeepConvLSTM model

In [6]:
from DeepConvLSTM_model import DeepConvNet
from Validation_model import Net
net = DeepConvNet(in_channels = 3, hidden_size = 256, output_size = 7)
model = Net(model = net, num_classes = 7, classes_weight = torch.tensor(train_weight, dtype = torch.float))

trainer = pl.Trainer(gpus=-1,
                     max_epochs=20,
                    #  limit_train_batches=0.1,
                    #  limit_val_batches=0.1,
                     log_every_n_steps = 200,
                     callbacks = [EarlyStopping(monitor = "val_f1_score", patience = 3, mode = "max")],
                     logger = TensorBoardLogger(save_dir = 'Validation_LSTM_logs/', name = "PAMAP2"),
                     )
trainer.fit(model, train_iter, val_iter)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | DeepConvNet      | 3 M   
1 | criterion | CrossEntropyLoss | 0     


1

## Validation

In [ ]:
from TransformerClassifier import TransformerClassifier
val_model = TransformerClassifier(in_channels = 27, output_size = 7, d_model = 100, nhead = 1, dim_feedforward = 4096,)
# val_model = DeepConvNet(hidden_size = 256)
# state_dict = torch.load("logs/PAMAP2_LSTM_model/version_2/checkpoints/epoch=8.ckpt")["state_dict"]
state_dict = torch.load("val_model_transformer.ckpt")["state_dict"]
state_dict = helper.remove_prefix_from_dict("model.", state_dict)
val_model.load_state_dict(state_dict, strict = False)
val_model.eval()
crit = nn.CrossEntropyLoss()

for batch in val_iter:
    data = batch["data"]
    label = batch["label"]
    output = val_model(data)
    val_loss = crit(output, label)
    print("Loss : ", val_loss, "\t\t F1 score : ", f1(output, label, num_classes = 7 average = "micro"), "\n")

## Training Transformer classifier

In [11]:
from TransformerClassifier import TransformerClassifier
from Validation_model import Net
net = TransformerClassifier(in_channels = 27, output_size = 7, d_model = 100, nhead = 1, dim_feedforward = 2048,)
model = Net(model = net, num_classes = 7, classes_weight = torch.tensor(train_weight, dtype = torch.float))

trainer = pl.Trainer(gpus=-1,
                     max_epochs=100,
                    #  limit_train_batches=0.1,
                    #  limit_val_batches=0.1,
                     log_every_n_steps = 200,
                     callbacks = [EarlyStopping(monitor = "val_f1_score", patience = 3, mode = "max")],
                     logger = TensorBoardLogger(save_dir = 'Validation_transformer_logs/', name = "PAMAP2"),
                     )
trainer.fit(model, train_iter, val_iter)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                  | Params
----------------------------------------------------
0 | model     | TransformerClassifier | 984 K 
1 | criterion | CrossEntropyLoss      | 0     


1

## Saved Model Score
Location : logs/PAMAP2_LSTM/version_2/checkpoints/epoch=8.ckpt

F1 Score : 96.21%

Loss     : 0.1168

## Parameters
LR           : 0.0001

Optimizer    : Adamax

Epochs       : 8

LR Scheduler : None

hidden_size : 256

## Saved Model Score
Location : logs/PAMAP2_transformer/version_11/checkpoints/epoch=49.ckpt

F1 Score : 96.96%

Loss     : 0.1113

## Parameters
LR           : 0.0001

Optimizer    : Adamax

Epochs       : 49

LR Scheduler : None

in_channels = 27, output_size = 7, d_model = 100, nhead = 1, dim_feedforward = 2048, 